# Oasis Reinsurance Test Tool v0.6

This notebook allows example reinsurance structures to be input in OED format and ran against the development version of the Oasis finanical engine. 

## Included
* Basic direct insurance modelling (not the complete engine included in Flamingo)
    * Buildings, Other buildings contents and BI
    * Blanket limit
    * Blanket deductible
    * Site limit
    * Site deductible 
* FAC treaties
* Quota share
* Surplus share
* Per-risk
* Cat XL treaties

## Not included
* Attachement basis
* Aggregate terms
* Multiple portfolios in a single set of input files

## Test cases

A selection of test cases can be found in the examples folder.

* simple_loc_FAC: Single location level fac.
* simple_pol_FAC: Single policy level fac.
* simple_acc_FAC: Single account level fac.
* multiple_FAC: Multiple facs at sameinuring level.
* simple_QS: Single quota share with no risk limits.
* loc_SS: Single surplus share at location level.
* pol_SS: Single surplus share at location level.
* acc_SS: Single surplus share at location level.
* multiple_SS: Multiple surplus shares at same inuring level.
* loc_limit_QS: Single quota share with location level risk limits.
* pol_limit_QS: Single quota share with policy level risk limits.
* acc_limit_QS: Single quota share with account level risk limits.
* multiple_QS_1:  Two quota shares at same inuring level.
* multiple_QS_2:  Two quota shares at different inuring levels.
* simple_CAT_XL: Single cat XL.
* multiple_CAT_XL: Two cat XLs at different inuring levels.


## Input files 

Files must be named the following:
* account.csv
* location.csv
* ri_info.csv
* ri_scope.csv


## Validation Rules
* Risk levels cannot be mixed in a single reinsurance scope
* Values in the scope file must link to rows in ACC/LOC exposure file set.
* QS always has non-specific scope
* SS always has specific scope
* Reinsurance types cannot be combined in an inuring layer

In [1]:
# Step 1 - environment and Python package setup
%config IPCompleter.greedy=True

# Standard Python libraries
import json
import os
import sys
!{sys.executable} -m pip uninstall -y oasislmf && pip install -v git+ssh://git@github.com/OasisLMF/OasisLMF.git@feature/mdk-reinsurance-integration#egg=oasislmf

# 3rd party Python libraries
import jupyter_helper
import pandas as pd    
import six
    
# Oasis imports
import oasislmf._data as static_data 
from oasislmf.exposures import oed
from oasislmf.exposures.reinsurance_layer import (
    create_xref_description,
    generate_files_for_reinsurance,
)
from oasislmf.utils.deterministic_loss import (
    generate_oasis_files,
    apply_fm,
)
import reinsurance_tester



Skipping oasislmf as it is not installed.
Created temporary directory: /private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-ephem-wheel-cache-kaz3i2at
Created temporary directory: /private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-req-tracker-5u8620i6
Created requirements tracker '/private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-req-tracker-5u8620i6'
Created temporary directory: /private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-install-c5s5irhg
  Cloning ssh://git@github.com/OasisLMF/OasisLMF.git (to revision feature/mdk-reinsurance-integration) to /private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-install-c5s5irhg/oasislmf
  Running command git clone -q ssh://git@github.com/OasisLMF/OasisLMF.git /private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-install-c5s5irhg/oasislmf
  Running command git show-ref feature/mdk-reinsurance-integration
  ccd70eae4b7ef9e593330d037172dbacd3829e8c refs/remotes/origin/feature/mdk-reinsurance-in

    Found link https://files.pythonhosted.org/packages/75/27/82da3fa4ea7a8c3526c48eaafe427352ff9c931633b917c2251826a43697/requests-2.19.0.tar.gz#sha256=cc408268d0e21589bcc2b2c248e42932b8c4d112f499c12c92e99e2178a6134c (from https://pypi.org/simple/requests/) (requires-python:>=2.6, !=3.0.*, !=3.1.*, !=3.2.*, !=3.3.*), version: 2.19.0
    Found link https://files.pythonhosted.org/packages/65/47/7e02164a2a3db50ed6d8a6ab1d6d60b69c4c3fdf57a284257925dfc12bda/requests-2.19.1-py2.py3-none-any.whl#sha256=63b52e3c866428a224f97cab011de738c36aec0185aa91cfacd418b5d58911d1 (from https://pypi.org/simple/requests/) (requires-python:>=2.6, !=3.0.*, !=3.1.*, !=3.2.*, !=3.3.*), version: 2.19.1
    Found link https://files.pythonhosted.org/packages/54/1f/782a5734931ddf2e1494e4cd615a51ff98e1879cbe9eecbdfeaf09aa75e9/requests-2.19.1.tar.gz#sha256=ec22d826a36ed72a7358ff3fe56cbd4ba69dd7a6718ffd450ff0e9df7a47ce6a (from https://pypi.org/simple/requests/) (requires-python:>=2.6, !=3.0.*, !=3.1.*, !=3.2.*, !=3.3.*

    Skipping link https://files.pythonhosted.org/packages/ac/0a/38fd730426fb1734ac8d6ba80b24457fb5626fc7941705f5276424a8ac34/Shapely-1.6b2-cp34-cp34m-manylinux1_x86_64.whl#sha256=6e044ea1d8a2463ccd632cc2374fe1a81a1fe90080b5425b0b0115a15b9abb40 (from https://pypi.org/simple/shapely/); it is not compatible with this Python
    Skipping link https://files.pythonhosted.org/packages/e2/6e/2d547912eddeaa8f19f8359ec90e594eb3c45317d5b21a6774b12c9eae70/Shapely-1.6b2-cp35-cp35m-macosx_10_6_intel.whl#sha256=9629bfd4d1ac0cc7878ceaefdb0a3b4934675035faef5b27b338cb7efcbbbe7e (from https://pypi.org/simple/shapely/); it is not compatible with this Python
    Skipping link https://files.pythonhosted.org/packages/88/82/43c71330b0248627cd7509b8ca4d681ddc97c708ea32bb2d71554a9ee0d7/Shapely-1.6b2-cp35-cp35m-manylinux1_x86_64.whl#sha256=b11dcb9caf5f798bd1116842b435581670c65c468e7d1040d6993f5310fda4b8 (from https://pypi.org/simple/shapely/); it is not compatible with this Python
    Found link https://files.py

  Looking up "https://files.pythonhosted.org/packages/c8/7f/a47c3899460e8b99130048b398ab6e16945ed7f71df60ec7bf816c74128a/Shapely-1.6.4.post1-cp36-cp36m-macosx_10_9_intel.macosx_10_9_x86_64.whl" in the cache
  Current age based on date: 18752611
  Ignoring unknown cache-control directive: immutable
  Freshness lifetime from max-age: 365000000
  The response is "fresh", returning cached response
  365000000 > 18752611
  Using cached https://files.pythonhosted.org/packages/c8/7f/a47c3899460e8b99130048b398ab6e16945ed7f71df60ec7bf816c74128a/Shapely-1.6.4.post1-cp36-cp36m-macosx_10_9_intel.macosx_10_9_x86_64.whl
  Added shapely==1.6.4.post1 from https://files.pythonhosted.org/packages/c8/7f/a47c3899460e8b99130048b398ab6e16945ed7f71df60ec7bf816c74128a/Shapely-1.6.4.post1-cp36-cp36m-macosx_10_9_intel.macosx_10_9_x86_64.whl#sha256=2222eba7f461f4b3f406b569727561f300ecd57e31e01b49ffd327dc4cbda632 (from oasislmf) to build tracker '/private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-req-tr

  Created temporary directory: /private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-wheel-svogwpl4
  Running setup.py bdist_wheel for oasislmf ...   Destination directory: /private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-wheel-svogwpl4
  Running command /Library/Frameworks/Python.framework/Versions/3.6/bin/python3 -u -c "import setuptools, tokenize;__file__='/private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-install-c5s5irhg/oasislmf/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-wheel-svogwpl4 --python-tag cp36
  /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/distutils/dist.py:261: UserWarning: Unknown distribution option: 'long_description_content_type'
    warnings.warn(msg)
  running bdist_wheel
  The [wheel] section is deprecated. Use [bdist_wheel] instead.
  running bu

  /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/distutils/dist.py:261: UserWarning: Unknown distribution option: 'long_description_content_type'
    warnings.warn(msg)
  running clean
  removing 'build/lib' (and everything under it)
  'build/bdist.macosx-10.6-intel' does not exist -- can't clean it
  removing 'build/scripts-3.6' (and everything under it)
  removing 'build'
Failed to build oasislmf
jupyter-console 6.0.0 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.15 which is incompatible.
  Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Created temporary directory: /private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-uninstall-q_a7i56_
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/requests-2.18.4.dist-info/DESCRIPTION.rst
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-p

      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/shapely/_buildcfg.py
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/shapely/affinity.py
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/shapely/algorithms/__init__.py
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/shapely/algorithms/__pycache__/__init__.cpython-36.pyc
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/shapely/algorithms/__pycache__/cga.cpython-36.pyc
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/shapely/algorithms/cga.py
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/shapely/coords.py
  


  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Created temporary directory: /private/var/folders/vx/3ts13s190k50b2gzt_6zyp0m0000gn/T/pip-uninstall-cr73altc
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/__pycache__/six.cpython-36.pyc
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/six-1.12.0.dist-info/INSTALLER
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/six-1.12.0.dist-info/LICENSE
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/six-1.12.0.dist-info/METADATA
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/six-1.12.0.dist-info/RECORD
      Removing file or directory /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-package

ModuleNotFoundError: No module named 'qgrid'

In [ ]:
jupyter_helper.file_uploader('examples/uploaded')

FileUploadWidget(label='Upload .CSV file', _dom_classes=('widget_item', 'btn-group'))

In [ ]:
# Step 2 - load the OED for a worked example, in this case FM Test Case 23.
# Note that only the currently used fields are shown unless show_all is set to True. 
oed_dir = os.path.abspath('examples/loc_SS')

# Account file
oed_account_file = os.path.join(oed_dir, "account.csv")
if not os.path.exists(oed_account_file):
    print("Path does not exist: {}".format(oed_account_file))
    exit(1)
account_df = pd.read_csv(oed_account_file)

# Location file
oed_location_file = os.path.join(oed_dir, "location.csv")
if not os.path.exists(oed_location_file):
    print("Path does not exist: {}".format(oed_location_file))
    exit(1)
location_df = pd.read_csv(oed_location_file)

(
    ri_info_df, 
    ri_scope_df, 
    do_reinsurance
) = oed.load_oed_dfs(oed_dir, show_all=False)

In [ ]:
# Step 3 - view/edit the account data. 
account_grid = jupyter_helper.show_df(account_df)
account_grid

In [ ]:
# Step 4 - view/edit the location data.
location_grid = jupyter_helper.show_df(location_df)
location_grid

In [ ]:
# Step 5 - view/edit the reinsurance info data.
ri_info_grid = jupyter_helper.show_df(ri_info_df)
ri_info_grid

In [ ]:
# Step 6 - view/edit the reinsurance scope data.
ri_scope_grid = jupyter_helper.show_df(ri_scope_df)
ri_scope_grid

In [ ]:
# Step 7 - Show mapping between Location numbers and item_id.
#direct_layer = reinsurance_tester.DirectLayer(account_df, location_df)
#direct_layer.generate_oasis_structures()
#direct_layer.report_item_ids()

data_fp = static_data.__path__._path[0]
with io.open(os.path.join(data_fp, 'canonical-oed-loc-profile.json'), 'r', encoding='utf-8') as f1:
    with io.open(os.path.join(data_fp, 'canonical-oed-acc-profile.json'), 'r', encoding='utf-8') as f2:
        with io.open(os.path.join(data_fp, 'fm-oed-agg-profile.json'), 'r', encoding='utf-8') as f3:
                cep = json.load(f1)
                cap = json.load(f2)
                # Agg profile keys are strings, so str -> int conversion is required
                fmap = {int(k):v for k, v in six.iteritems(json.load(f3))}            

oasis_files =  generate_oasis_files(
    os.path.abspath('run_reinsurance'),
    oed_location_file,
    os.path.join(data_fp, 'MappingMapToOED_CanLocA.xslt'),
    oed_account_file,
    os.path.join(data_fp, 'MappingMapToOED_CanAccA.xslt'),
    cep,
    cap
    fmap    
)

In [ ]:
# Step 8 - pick up any edits in the grid before running the analysis
account_df = account_grid.get_changed_df()
location_df = location_grid.get_changed_df()
ri_info_df = ri_info_grid.get_changed_df()
ri_scope_df = ri_scope_grid.get_changed_df()

In [ ]:
# Step 9 - run the OED data though the Oasis Financial Module and output the losses by item at each inuring level.
net_losses = reinsurance_tester.run_test('run_reinsurance', account_df, location_df, ri_info_df, ri_scope_df, loss_factor=1.0, do_reinsurance=do_reinsurance)
print("Ran {} inuring layers".format(len(net_losses) - 1))
print("Losses for:")
for key in net_losses.keys():
    print("\t{}".format(key))

In [ ]:
# Step 10 - view the direct losses.
key = 'Direct'
net_losses[key]

In [ ]:
# Step 11 - view the losses for the first inuring layer.
key = 'Inuring priority:1 - Risk level:SEL'
net_losses[key]

## Optional Steps


In [ ]:
#Automated testing on reinsurance examples
!pytest -v

In [ ]:
# Write outputs to CSV for download.
for (description, net_loss) in net_losses.items():
    filename = "output_{}.csv".format(description.replace(' ', '_'))
    net_loss.to_csv(filename, index=False)